In [ ]:
import identification_py2 as ident_tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import control as con
import glob #for returning files having the specified path extension
import statistics as stats
import os #checking for empty file
%pylab inline

###### Passing all the data into arrays 

In [ ]:
task_first        = sorted(glob.glob('step_log_new/*/*task*.log')) #corresponds to .log files that has data related to the first position
control_first     = sorted(glob.glob('step_log_new/*/*control*.log'))
task_remaining    = sorted(glob.glob('step_log_new/*/*task*.log.*')) #corresponds to remaining log.'n' files
control_remaining = sorted(glob.glob('step_log_new/*/*control*.log.*'))
task              = sorted(task_first + task_remaining) #set of all task_velocity logs
control           = sorted(control_first + control_remaining) #set of all control logs
observations      = len(task_first) #total number of experiments conducted/observations taken
positions         = int(len(task) / observations) #number of points in the given task space
task_full         = [] #A task_velocity list whose each element is a list of similar log files i.e from the same position
control_full      = [] #A control_output list whose each element is a list of similar log files i.e from the same position

for i in range(0, positions):
    task_full.append([])
    control_full.append([])
    for j in range(0, observations):
        task_full[i].append(task[i + (j * positions)])
        control_full[i].append(control[i + (j * positions)])

count = 0 #counter that returns the number of empty files
for i in range(0, positions):
    for j in range(0, observations):
        if os.stat(task_full[i][j]).st_size == 0:
            count = count + 1
            
for i in range(0, positions):
    for j in range(0, observations-count):
        if os.stat(task_full[i][j]).st_size == 0:
            del(task_full[i][j])
            del(control_full[i][j])
            
# Reading all the data into a dataframe array
df_ist_soll = []
model_estimations = []

for i in range(0, positions):
    df_ist_soll.append([])
    model_estimations.append([])
    for j in range(0, observations):
        try:
            # initialize result dictionary
            model_estimations[i].append({})
            df_ist_soll[i].append(ident_tools.batch_read_data(control_full[i][j], task_full[i][j]))
        except:
            print "Some Error occured and was ignored"
            continue
            
print "############################### INFO #########################################"
print "df_ist_soll contains all reference and actual data per each position and each experiement"
print
print "df_ist_soll[position][experiement] -> pandas.Dataframe{'x_ist', 'x_soll'}"
print "model_estimations[position][experiement] -> dict{'delay', 'scale_factor', ...}"
print "############################### INFO #########################################"


###### Manually changing the setpoint 

In [ ]:
#for i in range(0, observations):
#    df_ist_soll[0][i].x_soll[df_ist_soll[0][i].x_soll > 0] = 0.15
#    df_ist_soll[3][i].x_soll[df_ist_soll[3][i].x_soll > 0] = 0.17

###### Displaying all the observations

###### Smoothing using Savgol filter and normalization of data

In [ ]:
for i, df_pos in enumerate(df_ist_soll):
    for j, df_exp in enumerate(df_pos):
        
        # The following expects the unit step to be single and alone per experiement!
        if len(df_exp.x_soll.value_counts()) > 2:
            print "WARNING! - Data in Position {0}, Experiement {1} (df_ist_soll[{0}][{1}])".format(i, j)
            print "has more than {} different values". format(len(df_exp.x_soll.value_counts()))
            print "!!!! This could lead to wrong calculations!!!!"
            print "!!!! Target values should be either Zero (beginning) or step height (until end)!!!!"
            print df_exp.x_soll.value_counts()

        factor = 1.0/max(df_exp.x_soll)
        model_estimations[i][j]['step_height'] = max(df_exp.x_soll)
        model_estimations[i][j]['scale_factor'] = factor
        df_ist_soll[i][j] = pd.concat(
            [df_ist_soll[i][j],
            pd.DataFrame(
                factor*np.array(df_exp.x_ist),
                columns=['normalized'],
                index=df_exp.index)],
            axis=1)
        
        for order in range(1, 4):
            df_ist_soll[i][j] = pd.concat(
                [df_ist_soll[i][j],
                pd.DataFrame(
                    ident_tools.smooth(factor*np.array(df_exp.x_ist), order),
                    columns=['smooth_{}'.format(order)],
                    index=df_exp.index)],
                axis=1)

In [ ]:
df_ist_soll[0][0].plot()
print df_ist_soll[0][0].head()

print model_estimations[0][0]

###### PT1 + PT2 Estimation 

In [ ]:
for i, df_pos in enumerate(df_ist_soll):
    for j, df_exp in enumerate(df_pos):
        
        for order in range(1,4):
            # PT1 Estimations
            tf, _, _, delay, time_constant, steady_state, tf_without_delay =\
                ident_tools.pt1(np.array(df_exp['smooth_{}'.format(order)]), np.array(df_exp.index))
            pt1_dict = {
                'tf_delay': tf,
                'tf': tf_without_delay,
                'delay': delay,
                'time_constant': time_constant,
                'steady_state': steady_state
            }
            model_estimations[i][j]['pt1_smooth_{}'.format(order)] = pt1_dict
        
        for order in range(1,4):
            try:
                tf, _, _, delay, time_constant, steady_state, zeta, tf_without_delay =\
                    ident_tools.pt2(np.array(df_exp['smooth_{}'.format(order)]), np.array(df_exp.index))

                pt2_dict = {
                    'tf_delay': tf,
                    'tf': tf_without_delay,
                    'delay': delay,
                    'time_constant': time_constant,
                    'steady_state': steady_state,
                    'zeta': zeta,
                }
                model_estimations[i][j]['pt2_smooth_{}'.format(order)] = pt2_dict
            except:
                print "PT2 Estimation failed (smoothed_{2})! Position {0}, Experiement {1}; df_ist_soll[{0}][{1}]".format(i, j, order)
                #model_estimations[i][j]['pt2_smooth_{}'.format(order)] = None

        try:
            tf, _, _, delay, time_constant, steady_state, zeta, tf_without_delay =\
                ident_tools.pt2(np.array(df_exp['normalized']), np.array(df_exp.index))

            pt2_dict = {
                'tf_delay': tf,
                'tf': tf_without_delay,
                'delay': delay,
                'time_constant': time_constant,
                'steady_state': steady_state,
                'zeta': zeta,
            }
            model_estimations[i][j]['pt2'] = pt2_dict
        except:
            print "PT2 Estimation failed (normalized)! Position {0}, Experiement {1}; df_ist_soll[{0}][{1}]".format(i, j)
            #model_estimations[i][j]['pt2'] = None

In [ ]:
print model_estimations[0][0].keys()
print model_estimations[0][0]['pt2_smooth_2'].keys()

t0, y0 = con.step_response(model_estimations[0][0]['pt2_smooth_2']['tf'])
t1, y1 = con.step_response(model_estimations[0][0]['pt1_smooth_1']['tf_delay'])
t2, y2 = con.step_response(model_estimations[0][0]['pt2_smooth_2']['tf_delay'])

plot(t0+model_estimations[0][0]['pt2_smooth_2']['delay'], y0, label='shifted pt2')
#plot(t1, y1, label='pt1d')
plot(t2, y2, label='pt2d')
plot(df_ist_soll[0][0]['normalized'], label='data')
plt.legend()

In [ ]:
# TODO:
# Average values over position and experiement 

In [ ]:
def from_all_experiements(pos, est_sys_key, paramerter_key):
    res = []
    for j, exp in enumerate(model_estimations[pos]):
        if est_sys_key in model_estimations[pos][j]:
            res.append(model_estimations[pos][j][est_sys_key][paramerter_key])
    return res

def from_every_experiement(est_sys_key, paramerter_key):
    res = []
    for i, pos in enumerate(model_estimations):
        for j, exp in enumerate(pos):
            if est_sys_key in pos[j]:
                res.append(exp[est_sys_key][paramerter_key])
    return res

print from_all_experiements(0, 'pt1_smooth_2', 'steady_state')
print
print from_every_experiement('pt1_smooth_2', 'steady_state')

###### State Space Parameters of all the ideal models in a textfile

###### Model Validation